# new imp score 뽑아보자

In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
files = glob.glob('./Biz 액터/*.csv')

In [3]:
dfs = []
for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

In [4]:
imp = pd.concat(dfs)

In [5]:
imp.head()

,Unnamed: 0,action,sent score,imp score,opp score,Column1
0,0.0,Cluster1_actor1_action1,6.3717,5.4845,5.4845,NaN
1,1.0,Cluster1_actor1_action2,5.4528,5.7569,6.0610,NaN
2,2.0,Cluster1_actor1_action3,5.3733,5.1684,5.1684,NaN
3,3.0,Cluster1_actor1_action4,5.1829,5.5315,5.8801,NaN
4,4.0,Cluster1_actor1_action5,5.4795,5.2904,5.2904,NaN


In [6]:
imp = imp.drop(columns=['Unnamed: 0', 'Column1'])

In [7]:
imp = imp[imp.action != 'Cluster1_actor2_action4']
imp.head()

,action,sent score,imp score,opp score
0,Cluster1_actor1_action1,6.3717,5.4845,5.4845
1,Cluster1_actor1_action2,5.4528,5.7569,6.0610
2,Cluster1_actor1_action3,5.3733,5.1684,5.1684
3,Cluster1_actor1_action4,5.1829,5.5315,5.8801
4,Cluster1_actor1_action5,5.4795,5.2904,5.2904


In [8]:
imp

,action,sent score,imp score,opp score
0,Cluster1_actor1_action1,6.3717,5.4845,5.4845
1,Cluster1_actor1_action2,5.4528,5.7569,6.0610
2,Cluster1_actor1_action3,5.3733,5.1684,5.1684
3,Cluster1_actor1_action4,5.1829,5.5315,5.8801
4,Cluster1_actor1_action5,5.4795,5.2904,5.2904
...,...,...,...,...
1,Cluster6_actor2_action2,7.4258,9.0226,10.6194
2,Cluster6_actor2_action3,5.7356,3.6447,3.6447
3,Cluster6_actor2_action4,5.5389,5.7044,5.8699
4,Cluster6_actor2_action5,5.6751,4.1857,4.1857


In [9]:
imp['sent score'].mean()

5.636809677419355

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
imp["scaled_imp_score"] = scaler.fit_transform(imp[['imp score']])
imp["scaled_sent_score"] = scaler.fit_transform(imp[['sent score']])
mean_sent = imp['sent score'].mean()

In [20]:
import math

def sigmoid(x):
    sig = 1 / (1 + math.exp(-x))
    return sig

imp["scaled_imp_score"] = imp["scaled_imp_score"].apply(lambda x : sigmoid(x*2) * 10)
imp["scaled_sent_score"] = imp["scaled_sent_score"].apply(lambda x : sigmoid(x*2) * 10)
mean_sent = sigmoid(mean_sent*2) * 10

In [15]:
imp

,action,sent score,imp score,opp score,scaled_imp_score,scaled_sent_score
0,Cluster1_actor1_action1,6.3717,5.4845,5.4845,9.999899,10.000000
1,Cluster1_actor1_action2,5.4528,5.7569,6.0610,9.999996,9.994247
2,Cluster1_actor1_action3,5.3733,5.1684,5.1684,9.994163,9.984121
3,Cluster1_actor1_action4,5.1829,5.5315,5.8801,9.999944,9.872221
4,Cluster1_actor1_action5,5.4795,5.2904,5.2904,9.998745,9.995968
...,...,...,...,...,...,...
1,Cluster6_actor2_action2,7.4258,9.0226,10.6194,10.000000,10.000000
2,Cluster6_actor2_action3,5.7356,3.6447,3.6447,5.548401,9.999886
3,Cluster6_actor2_action4,5.5389,5.7044,5.8699,9.999993,9.998208
4,Cluster6_actor2_action5,5.6751,4.1857,4.1857,7.054880,9.999735


In [13]:
imp.reset_index(drop=True, inplace=True)

In [14]:
opp_score = []

for i in imp.index:
    opp = imp.scaled_imp_score[i]+max(imp.scaled_imp_score[i]- imp['scaled_sent_score'][i],0)
    opp_score.append(round(opp,4))
print('★액션의 기회영역 점수(x10 해서 CAM에 입력):',opp_score)

★액션의 기회영역 점수(x10 해서 CAM에 입력): [5.7499, 10.9417, 4.2237, 9.9193, 5.0748, 4.2833, 5.194, 6.052, 15.8499, 16.9806, 5.5846, 3.8628, 3.1237, 3.2454, 3.2873, 7.6186, 6.8146, 7.34, 3.0143, 5.1164, 6.9289, 5.6803, 6.795, 4.2405, 4.8047, 4.8797, 5.3487, 13.94, 7.091, 13.7674, 3.1114, 6.2372, 4.92, 5.7315, 2.9608, 4.0765, 8.3603, 5.2232, 4.4198, 9.2521, 4.1199, 3.4134, 12.3063, 5.5685, 5.9687, 6.7278, 4.7143, 3.2913, 6.1457, 11.5068, 11.8223, 6.1016, 8.0077, 5.5497, 4.2692, 16.6668, 0.0034, 10.0623, 0.1101, 9.8061, 0.4368, 0.4687]


In [15]:
imp['scaled_opp_score'] = opp_score

In [16]:
imp

,action,sent score,imp score,opp score,scaled_imp_score,scaled_sent_score,scaled_opp_score
0,Cluster1_actor1_action1,6.3717,5.4845,5.4845,5.749890,8.883671,5.7499
1,Cluster1_actor1_action2,5.4528,5.7569,6.0610,7.335887,3.730036,10.9417
2,Cluster1_actor1_action3,5.3733,5.1684,5.1684,3.722805,3.221879,4.2237
3,Cluster1_actor1_action4,5.1829,5.5315,5.8801,6.046445,2.173590,9.9193
4,Cluster1_actor1_action5,5.4795,5.2904,5.2904,4.491373,3.907897,5.0748
...,...,...,...,...,...,...,...
57,Cluster6_actor2_action2,7.4258,9.0226,10.6194,9.999276,9.936269,10.0623
58,Cluster6_actor2_action3,5.7356,3.6447,3.6447,0.110123,5.692588,0.1101
59,Cluster6_actor2_action4,5.5389,5.7044,5.8699,7.059799,4.313507,9.8061
60,Cluster6_actor2_action5,5.6751,4.1857,4.1857,0.436785,5.269916,0.4368


In [40]:
imp.to_csv('KB_감성_기회점수.csv', encoding='utf-8-sig')